In [0]:
WITH RECURSIVE params AS (
     SELECT CAST(:MinSpend AS DECIMAL(18,6)) as medical_spend
    
    UNION ALL
    
    SELECT CAST(medical_spend + :SpendIncrement AS DECIMAL(18,6))
    FROM params
    WHERE medical_spend + :SpendIncrement <= :MaxSpend

), plans AS (
    SELECT *
    FROM (
        VALUES
            ('Plan A', 1050, 3300,  8000, 0.90),
            ('Plan B',  550, 6600, 13300, 0.90),
            ('Plan C',  350,10000, 13300, 0.80)
    ) AS t(plan_name, monthly_premium, deductible, out_of_pocket_max, coinsurance_rate) 
), calc AS (
    SELECT
        p.*,
        pr.medical_spend,
        CASE
            WHEN pr.medical_spend <= p.deductible
                THEN pr.medical_spend
            ELSE LEAST(
                    p.out_of_pocket_max,
                    p.deductible + (1 - p.coinsurance_rate) * (pr.medical_spend - p.deductible)
                 )
        END AS event_oop
    FROM plans p
    CROSS JOIN params pr
)

SELECT
    plan_name,
    monthly_premium,
    monthly_premium * 12 AS annual_premium,
    medical_spend,
    event_oop,
    (monthly_premium * 12) + event_oop AS total_annual_cost
FROM calc
ORDER BY plan_name,medical_spend;